In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os
print("done")

done


In [2]:
lr =  1e-4 # learing RATE 0.0001
bs = 32  
epochs= 20 

In [3]:
directory="C:\\Users\\dataset"
category=["with_mask", "without_mask"]
print(category)

['with_mask', 'without_mask']


In [4]:
#PREPROCESSING

data = []  # images
label = []  # labels (with,without_mask)
for i in category:
    path= os.path.join(directory, i)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224, 224)) # load the image and the resize value
        image = img_to_array(image) # convert to numpy array
        image = preprocess_input(image)  
        #print(image)
        data.append(image)
        label.append(i)
# TO convert the labels with_mask and without_mask as numberical (one hot encoding)
lb = LabelBinarizer()
label= lb.fit_transform(label)
label=to_categorical(label)
print("done")
#to convert into a numpy array
data = np.array(data,dtype = "float32")
label = np.array (label)
#print(data)
#print(label)

done


In [5]:
(trainX,testX,trainY,testY) = train_test_split(data,label,test_size= 0.2,stratify = label,  random_state = 42) 


In [6]:
#Data augmentation
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest" )

In [ ]:
#model
basemodel = MobileNetV2(input_shape=(224, 224, 3),weights ="imagenet", include_top = False, input_tensor =Input(shape=(224,224,3)))
print(basemodel)

In [ ]:
headmodel = basemodel.output
headmodel = AveragePooling2D(pool_size = (7,7))(headmodel)
headmodel = Flatten(name="flatten")(headmodel)
headmodel = Dense(128, activation = "relu")(headmodel)
headmodel = Dropout(0.5)(headmodel)
headmodel = Dense(2,activation="softmax")(headmodel)
print("done")

In [ ]:
model= Model(inputs = basemodel.input , outputs = headmodel)


In [ ]:
for layer in basemodel.layers:
    layer.trainable = False
#print(layer)

In [ ]:
opt = Adam(lr = lr, decay = lr / epochs)
model.compile(loss="binary_crossentropy", optimizer=opt,
              metrics=["accuracy"])


In [ ]:
H = model.fit(
    aug.flow(trainX, trainY, batch_size=bs),
    steps_per_epoch=len(trainX) // bs,
    validation_data=(testX, testY),
    validation_steps=len(testX) // bs,
    epochs=epochs)


In [ ]:
pred = model.predict(testX, batch_size=bs)
